In [ ]:
%%capture
import pandas as pd
from django_pandas.io import read_frame
from pathlib import Path
from dj_notebook import activate

plus = activate(dotenv_file="/Users/erikvw/source/edc_source/meta-edc/.env")
report_folder = Path("/Users/erikvw/Documents/ucl/protocols/meta3/reports/")
# output is suppressed -- normally would spew out all the edc loading messages


In [ ]:
from meta_prn.models import OnSchedule, OffSchedule, EndOfStudy, OffSchedulePostnatal, OffSchedulePregnancy, \
    OnScheduleDmReferral, OffStudyMedication
from meta_lists.models import OffstudyReasons
from meta_analytics.dataframes import GlucoseEndpointsByDate

In [ ]:
df_onschedule = read_frame(OnSchedule.objects.all(), verbose=True)
df_offschedule = read_frame(OffSchedule.objects.all(), verbose=True)
df_onschedule_preg = read_frame(OffSchedulePregnancy.objects.all(), verbose=True)
df_onschedule_postnatal = read_frame(OffSchedulePostnatal.objects.all(), verbose=True)
df_onschedule_dm = read_frame(OnScheduleDmReferral.objects.all(), verbose=True)
df_offstudy_medication = read_frame(OffStudyMedication.objects.all(), verbose=True)

df_eos = read_frame(EndOfStudy.objects.values("id", "subject_identifier",  "offstudy_datetime","offstudy_reason", "other_offstudy_reason", "action_item__action_type__name", "parent_action_item__action_type__name").all(), verbose=False)
df_eos.rename(columns={"offstudy_reason": "offstudy_reason_id", "action_item__action_type__name": "action_item_name", "parent_action_item__action_type__name": "parent_action_item_name"}, inplace=True)

df_offstudy_reasons = read_frame(OffstudyReasons.objects.all(), verbose=False)
df_offstudy_reasons.rename(columns={"id": "offstudy_reason_id", "name": "offstudy_reason", "display_name": "offstudy_reason_display"}, inplace=True)

In [ ]:
# merge df_eos with offstudyreasons
df_eos = df_eos.merge(df_offstudy_reasons[["offstudy_reason_id", "offstudy_reason", "offstudy_reason_display"]], on="offstudy_reason_id")


In [ ]:
# summarize offstudy
df_eos_summary = df_eos[["offstudy_reason_id", "offstudy_reason"]].value_counts().to_frame().reset_index().sort_values("offstudy_reason_id", ascending=True).reset_index(drop=True)
df_eos_summary.rename(columns={"count": "subtotal"}, inplace=True)
df_eos_summary["cumfreq"] = df_eos_summary.subtotal.cumsum()
df_eos_summary


* Need to validate that all subjects off study went off study medication  first.
* Action items are linked to this flow and block the EndOfStudy.save() if  EndOfStudy was submitted before this workflow was implemented.


In [ ]:
# df_eos[df_eos.offstudy_reason_id==11][["subject_identifier", "offstudy_reason_id", "other_offstudy_reason"]]
pd.merge(df_eos, df_offstudy_medication[["subject_identifier", "stop_date"]], on="subject_identifier", how="left")


In [ ]:
df_onschedule_preg

In [ ]:
df_dm = df_onschedule_dm[["subject_identifier"]].copy()
df_dm.reset_index(drop=True, inplace=True)
df_dm["dm"] =1

df_preg = df_onschedule_preg[["subject_identifier"]].copy()
df_preg["preg"] = 1
df_preg.reset_index(drop=True, inplace=True)
#
df_eos = df_eos[["subject_identifier", "offstudy_datetime", "offstudy_reason", "other_offstudy_reason"]].copy()
df_eos["eos"] = 1
df_eos.reset_index(drop=True, inplace=True)

df = df_onschedule[["subject_identifier", "onschedule_datetime"]].copy()
df["onschedule"] = 1
df.reset_index(drop=True, inplace=True)



In [ ]:
# df[df.offstudy_reason.isna()]

In [ ]:
df = df.merge(df_dm, on=["subject_identifier"], how="left", suffixes=("", "_y"))
df = df.merge(df_preg, on=["subject_identifier"], how="left", suffixes=("", "_y"))


In [ ]:
df_offschedule = df_offschedule[["subject_identifier", "offschedule_datetime"]].copy()
df_offschedule["offschedule"] = 1
df_offschedule.reset_index(drop=True, inplace=True)
df = df.merge(df_offschedule, on=["subject_identifier"], how="left", suffixes=("", "_y"))
df = df.merge(df_eos, on=["subject_identifier"], how="left", suffixes=("", "_y"))

In [ ]:
cls = GlucoseEndpointsByDate()
cls.run()

In [ ]:
df_endpoint = cls.df
df_endpoint = df_endpoint[(df_endpoint.endpoint==1)][["subject_identifier", "endpoint"]]
df = df.merge(df_endpoint, on=["subject_identifier"], how="left", suffixes=("", "_y"))

In [ ]:
df.fillna({"dm": 0, "preg":0, "offschedule":0, "eos": 0, "endpoint": 0}, inplace=True)

In [ ]:
df.dm.value_counts()

In [ ]:
df.preg.value_counts()

In [ ]:
df_export = df[(df.offschedule_datetime.notna()) & (df.eos==0) & ((df.dm==0) & (df.preg==0))][["subject_identifier", "onschedule_datetime", "offschedule_datetime", "onschedule", "offschedule", "dm", "preg", "endpoint", "eos"]].copy()
df_export.reset_index(drop=True, inplace=True)
df_export["offschedule_datetime"] = df_export.offschedule_datetime.dt.tz_localize(None)
df_export["offschedule_datetime"] = df_export.offschedule_datetime.dt.normalize()
df_export["onschedule_datetime"] = df_export.onschedule_datetime.dt.tz_localize(None)
df_export["onschedule_datetime"] = df_export.onschedule_datetime.dt.normalize()
df_export.sort_values(by=["offschedule_datetime"], inplace=True)
df_export.to_csv(report_folder / "offschedule_eos.csv", index=False)

In [ ]:
df

In [ ]:
df[df.endpoint==1]

In [ ]:
# DM Referrals onstudy/off study
df.groupby(by=["dm", "eos"]).size().reset_index(name="counts")

In [ ]:
df.groupby(by=["preg", "eos"]).size().reset_index(name="counts")

In [ ]:
df_eos[(df_eos.other_offstudy_reason != "")][["subject_identifier", "offstudy_reason", "other_offstudy_reason"]]

In [ ]:
df_eos.offstudy_reason.value_counts()

In [ ]:
df_eos = read_frame(EndOfStudy.objects.all(), verbose=False)
df_offstudy_reasons = read_frame(OffstudyReasons.objects.all(), verbose=False)
df_offstudy_reasons.rename(columns={"id": "offstudy_reason_id", "name": "offstudy_reason"}, inplace=True)
df_eos = df_eos.merge(df_offstudy_reasons["offstudy_reason_id", "offstudy_reason"], on="offstudy_reason_id")
df_eos.offstudy_reason.value_counts()